<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Deep_Q_Networks_(DQN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque

class DQN(nn.Module):
    def __init__(self):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

env = gym.make('CartPole-v1', new_step_api=True)
model = DQN()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()
memory = deque(maxlen=10000)
batch_size = 64
gamma = 0.99

def select_action(state, epsilon):
    if random.random() < epsilon:
        return env.action_space.sample()
    else:
        with torch.no_grad():
            return model(torch.tensor(state, dtype=torch.float32)).argmax().item()

for episode in range(500):
    state = env.reset() if isinstance(env.reset(), tuple) else env.reset()
    epsilon = max(0.01, 0.08 - 0.01 * (episode / 200))
    total_reward = 0
    done = False

    while not done:
        action = select_action(state, epsilon)
        next_state, reward, done, _, _ = env.step(action)
        total_reward += reward
        memory.append((state, action, reward, next_state, done))
        state = next_state

        if len(memory) >= batch_size:
            batch = random.sample(memory, batch_size)
            states, actions, rewards, next_states, dones = zip(*batch)
            states = torch.tensor(states, dtype=torch.float32)
            actions = torch.tensor(actions, dtype=torch.int64)
            rewards = torch.tensor(rewards, dtype=torch.float32)
            next_states = torch.tensor(next_states, dtype=torch.float32)
            dones = torch.tensor(dones, dtype=torch.float32)

            current_q = model(states).gather(1, actions.unsqueeze(1)).squeeze(1)
            max_next_q = model(next_states).max(1)[0]
            expected_q = rewards + (1 - dones) * gamma * max_next_q

            loss = criterion(current_q, expected_q)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    if episode % 10 == 0:
        print(f"Episode {episode}, total reward: {total_reward}")